In [72]:
## ALGORITMO k-opt / LI-KERNIGHAM ##

In [73]:
###############################################
###### ALGORITMO DEL VECINO MÁS PRÓXIMO #######
###############################################


In [74]:
## ABRIR DATASET ACTUALIZADO
import pickle

with open("dataset_4_actualizado.pkl", "rb") as f:
    datos = pickle.load(f)

#print(datos);print(datos.keys());print(datos["data"]);print(datos["distancias"]);print(datos["tiempos"]);print(datos["costes"])


In [75]:
## CREAR EL CONJUNTO DE NODOS ##
base_operativa_ficticia=-1
N = [i for i,d in enumerate(datos["data"]) if d['nodo']=="carga"] # nodos de carga
F = [i for i,d in enumerate(datos["data"]) if d['nodo']=="cargador"] # nodos cargador
V = list(range(len(datos["data"])))+[base_operativa_ficticia] # Nodos totales
base_operativa = [i for i,d in enumerate(datos["data"]) if d['nodo']=="parking"][0] 
descarga = [i for i,d in enumerate(datos["data"]) if d['nodo']=="descarga"][0] 

#print(V);print(N);print(F);print(base_operativa);print(descarga);print(base_operativa_ficticia)


In [76]:
# CREACIÓN DE ARRAYS
import numpy as np
c_0 = np.array(datos["costes"])
w = np.array([d["w"] for d in datos["data"]])
tiempos = np.array(datos["tiempos"])
distancias=np.array(datos["distancias"])

In [77]:
# La idea es elegir los cargadores por la distancia y luego sumarles el precio de carga. Pero como tenemos en cuenta esa diferencia de precio para la elección? Pues lo tomamos
# como si el coste de llegar a esos cargadores fuese, para los de menos coste, solo su coste real, y para los de más coste, su coste real + la diferencia de precio de carga.

# Vamos a actualizar el vector de costes.
# Tomaremos para los cargadores de ionity su coste real de llegar (por ser el mínimo)
# Tomaremos para los cargadores de Atlante la diferencia (9) de precio de carga + el coste total
c=[[0]*len(c_0) for _ in range(len(c_0))]
for i in range(len(c_0)):
    for j in range(len(c_0)):
        if i==9 or i==10: #cargadores de Atlante
            c[i][j]=c_0[i][j]+9
            if j!=i:
                c[j][i]=c_0[j][i]+9
            else: 
                c[j][i]=c_0[j][i]
        else:
            c[i][j]=c_0[i][j]
c = np.array(c)


In [78]:
# ALGORITMO DEL VECINO MÁS PRÓXIMO 
no_visitados = N.copy()+ F.copy()
cargadores= F.copy()  
ruta = [base_operativa] # La ruta comienza en la base operativa
posicion = base_operativa # Posición actual del vehículo
coste_total = 0
costes= c.copy()

while no_visitados:
    costes_posibles = {i: costes[posicion][i] for i in no_visitados}
    if not costes_posibles:
        break 

    # Encontrar el coste mínimo
    cost = min(costes_posibles.values())
    siguiente = min(costes_posibles, key=costes_posibles.get)

    # Si se selecciona un cargador, no se visitan más cargadores
    if siguiente in cargadores:
        no_visitados= [i for i in no_visitados if i not in cargadores]
        cost+= 183 # coste de  carga (a los de atlante ya se les sumó 9) 
    else: 
          no_visitados.remove(siguiente)

    # Actualizar la ruta
    coste_total += cost
    ruta.append(siguiente)

    # Actualizar la posición
    posicion = siguiente

# Ir a la descarga y volver a la base operativa
ruta.append(descarga)
coste_total += costes[posicion][descarga] 
ruta.append(base_operativa)
coste_total += costes[descarga][base_operativa] 
print("Ruta:",ruta) 
print("Coste de la distancia recorrida:", coste_total)


Ruta: [0, 1, 3, 5, 2, 4, 0]
Coste de la distancia recorrida: 211.34799999999998


In [79]:

# La ruta que hemos hecho no tiene en cuenta si se puede ir a un cargador entre la descarga y la base operativa
# Vamos a calcular esa ruta y ver si es mejor que la que tenemos
no_visitados_1 = N.copy()
ruta_1 = [base_operativa] # La ruta comienza en la base operativa
posicion_1 = base_operativa # Posición actual del vehículo
coste_total_1 = 0
costes_1= c.copy()

while no_visitados_1:
    costes_posibles_1 = {i: costes_1[posicion_1][i] for i in no_visitados_1}
    if not costes_posibles_1:
        break 

    # Encontrar el coste mínimo
    cost_1 = min(costes_posibles_1.values())
    siguiente_1 = min(costes_posibles_1, key=costes_posibles_1.get)

    # Actualizar la ruta
    coste_total_1 += cost_1
    ruta_1.append(siguiente_1)
    no_visitados_1.remove(siguiente_1)

    # Actualizar la posición
    posicion_1 = siguiente_1

# Ir a la descarga 
ruta_1.append(descarga)
coste_total_1 += costes_1[posicion_1][descarga] 

# Visitar un cargador
cargadores_1= F.copy()
costes_cargadores={i: costes_1[descarga][i] for i in cargadores_1}

# Encontrar el coste mínimo
cost_2 = min(costes_cargadores.values())
siguiente_1 = min(costes_cargadores, key=costes_cargadores.get)
coste_total_1 += cost_2 + 183
ruta_1.append(siguiente_1)

#Volver a la base operativa
ruta_1.append(base_operativa)
coste_total_1 += costes_1[descarga][base_operativa] 
print("Ruta:",ruta_1) 
print("Coste de la distancia recorrida:", coste_total_1)

Ruta: [0, 1, 3, 2, 4, 6, 0]
Coste de la distancia recorrida: 211.136


In [80]:
rutas={tuple(ruta):coste_total, tuple(ruta_1):coste_total_1}
ruta_optima = min(rutas, key=rutas.get)
print("Ruta óptima:", ruta_optima)
print("Coste mínimo:", rutas[ruta_optima])

Ruta óptima: (0, 1, 3, 2, 4, 6, 0)
Coste mínimo: 211.136


In [81]:
#########################################
###### ALGORITMO DE LIN-KERNIGHAN #######
#########################################

In [82]:
# FUNCION PARA CALCULAR COSTE DE UNA RUTA

import numpy as np

def calcular_coste(ruta, costes=c_0):
    coste = 0
    for i in range(len(ruta) - 1):
        coste += c_0[ruta[i], ruta[i+1]]+ 183
    return coste

In [83]:
# Ruta inicial (solución vecino más próximo)
ruta_actual = list(ruta_optima)
coste_actual = rutas[tuple(ruta_actual)]

print("Ruta inicial:", ruta_actual)
print("Coste inicial:", coste_actual)

# Fijamos el número máximo de iteraciones
max_iter = 100
iteracion = 0
mejora = True

# Iniciamos el algoritmo:
while mejora and iteracion < max_iter:
    mejora = False
    iteracion += 1 # Cada vuelta completa es una iteracion
    
    for j in range(1, len(ruta_actual) - 3): #siempre uno menos que i
        for i in range(1, len(ruta_actual)-2-j): # No cogemos la base operativa (inicial y final) ni el nodo de descarga
            nueva_ruta = ruta_actual.copy()
        
        # intercambiar el nodo i y el nodo i+j
            nueva_ruta[i], nueva_ruta[i+j] = nueva_ruta[i+j], nueva_ruta[i]
        
        # calcular nuevo coste
            nuevo_coste = calcular_coste(nueva_ruta, distancias)
        
        # Si mejora, la aceptamos
            if nuevo_coste < coste_actual:
                print("### Mejora encontrada en iteración" ,iteracion)
                print(".  Se intercambian los nodos", nueva_ruta[i],",", nueva_ruta[i+j], "(índices", i, ",", i+j , ").")
                print("   Nueva ruta:", nueva_ruta)
                print("   Nuevo coste:", nuevo_coste)
            
                ruta_actual = nueva_ruta
                coste_actual = nuevo_coste
                mejora = True
                 
        
# RESULTADO FINAL
print("Ruta final:", ruta_actual)
print("Coste final:", coste_actual)
print("Iteraciones realizadas:", iteracion)


Ruta inicial: [0, 1, 3, 2, 4, 6, 0]
Coste inicial: 211.136
Ruta final: [0, 1, 3, 2, 4, 6, 0]
Coste final: 211.136
Iteraciones realizadas: 1


In [84]:
# Como en ningún caso mejora vamos a intercambiar los dos ultimos nodos vistados
#  antes de llegar al nodo de descarga para ver si realmente mejoraría con otra
#  solución inicial

# Copiamos la ruta inicial original
ruta_actual = list(ruta_optima)

# Encontrar la posición del nodo 5 (descarga)
pos_descarga = ruta_actual.index(descarga)

# Intercambiar los dos nodos anteriores a 5 (si existen)
if pos_descarga >= 2:  
    ruta_actual[pos_descarga-1], ruta_actual[pos_descarga-2] = ruta_actual[pos_descarga-2], ruta_actual[pos_descarga-1]

print("Ruta inicial modificada:", ruta_actual)

Ruta inicial modificada: [0, 1, 2, 3, 4, 6, 0]


In [85]:
ruta_actual = ruta_actual
coste_actual =calcular_coste(ruta_actual, c_0)

print("Ruta inicial:", ruta_actual)
print("Coste inicial:", coste_actual)

# Fijamos el número máximo de iteraciones
max_iter = 100
iteracion = 0
mejora = True


# Iniciamos el algoritmo:
while mejora and iteracion < max_iter:
    mejora = False
    iteracion += 1 # Cada vuelta completa es una iteracion
    
    for j in range(1, len(ruta_actual) - 3): #siempre uno menos que i
        for i in range(1, len(ruta_actual)-2-j): # No cogemos la base operativa (inicial y final) ni el nodo de descarga
            nueva_ruta = ruta_actual.copy()
        
        # intercambiar el nodo i y el nodo i+j
            nueva_ruta[i], nueva_ruta[i+j] = nueva_ruta[i+j], nueva_ruta[i]
        
        # calcular nuevo coste
            nuevo_coste = calcular_coste(nueva_ruta, distancias)
            
        # Si mejora, la aceptamos
            if nuevo_coste < coste_actual:
                print("### Mejora encontrada en iteración" ,iteracion, ".")
                print(".  Se intercambian los nodos", nueva_ruta[i],",", nueva_ruta[i+j], "(índices", i, ",", i+j , ").")
                print("   Nueva ruta:", nueva_ruta)
                print("   Nuevo coste:", nuevo_coste)
            
                ruta_actual = nueva_ruta
                coste_actual = nuevo_coste
                mejora = True
                 
        
# RESULTADO FINAL
print("Ruta final:", ruta_actual)
print("Coste final:", coste_actual)
print("Iteraciones realizadas:", iteracion)


Ruta inicial: [0, 1, 2, 3, 4, 6, 0]
Coste inicial: 1124.4679999999998
### Mejora encontrada en iteración 1 .
.  Se intercambian los nodos 3 , 2 (índices 2 , 3 ).
   Nueva ruta: [0, 1, 3, 2, 4, 6, 0]
   Nuevo coste: 1124.296
Ruta final: [0, 1, 3, 2, 4, 6, 0]
Coste final: 1124.296
Iteraciones realizadas: 2
